# 回归模型



Predicting a continuous-valued attribute associated with an object.

Regression VS Classification
1. target variable is continuous or discrete。 区别在于结果是连续还是离散
2. evaluation metrics diff。另外评估方式也不同

很多模型都是既可以做分类，又可以做回归，差异很小。
本节课主要介绍下面三种模型在回归上的使用。

1. [SVR](http://www.svms.org/regression/SmSc98.pdf)
2. [Boosting Regression](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.31.314&rep=rep1&type=pdf)
3. [Random Forest](https://www.stat.berkeley.edu/~breiman/randomforest2001.pdf)

### 1. SVR

In [ ]:
class sklearn.svm.SVR(kernel='rbf', degree=3, gamma='auto', coef0=0.0, tol=0.001, C=1.0, 
                      epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("forestfires.csv")

In [2]:
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [4]:
# Onehotencoder
newDf = pd.get_dummies(data = df, columns=['month', 'day'])

In [5]:
newDf.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,month_nov,month_oct,month_sep,day_fri,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
xdata = newDf.drop('area', axis = 1)
ydata = newDf['area']

In [8]:
from sklearn.svm import SVR
svr = SVR(kernel = "linear", verbose = True)

In [9]:
svr.fit(xdata, ydata)

[LibSVM]

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=True)

In [11]:
svr.score(xdata, ydata)
# R^2
# 虽然是负数，只要去掉符号就可以了。官方说了是bug

-0.032413254922760926

**因为是SVM，我们试着把数据标准化后再进行处理，看看效果怎样**

In [13]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [14]:
xdata_ss = ss.fit_transform(xdata)

In [15]:
svr.fit(xdata_ss, ydata)

[LibSVM]

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=True)

In [16]:
svr.score(xdata_ss, ydata)

-0.032109399537486372

**结果发现并没有多大提升，反而有些下降**，这是为什么呢？

一般来说，做一个回归模型的时候，做标准化不太有意义。因为它有一个叫做截距的东西。对于一些，线性的，比如最小二成，这个截距会很好地解决量纲的问题。

但是做标准化是没有问题的。我们只是想说明，即使做了意义也不太大。

### 2. GBDT

In [17]:
from sklearn.ensemble import GradientBoostingRegressor

In [18]:
gbr = GradientBoostingRegressor(learning_rate = 0.01, n_estimators=200, subsample=0.7, max_depth=5)

In [19]:
# boosting是绝对不需要做标准化的
gbr.fit(xdata, ydata)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=5, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=0.7, verbose=0,
             warm_start=False)

In [20]:
gbr.score(xdata, ydata)

0.8803431556346426

score的结果同样都是R^2，但比SVR的效果好了很多。
一般来说集成模型都会比单个模型效果好很多。

下面的feature_importances 就是feature的重要性。因为树模型需要根据模型的重要性来进行选择。对应的就是xdata里的column.

比如说你做完了一个项目，你可以说发现哪几个因素对模型影响很大。或者把不重要的feature去掉。

In [22]:
xdata.columns

Index(['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain',
       'month_apr', 'month_aug', 'month_dec', 'month_feb', 'month_jan',
       'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov',
       'month_oct', 'month_sep', 'day_fri', 'day_mon', 'day_sat', 'day_sun',
       'day_thu', 'day_tue', 'day_wed'],
      dtype='object')

In [21]:
gbr.feature_importances_

array([  4.22637910e-02,   4.53662081e-02,   2.49178036e-02,
         1.28985559e-01,   3.90833546e-02,   6.85097568e-02,
         4.23770932e-01,   8.75160360e-02,   6.34827881e-02,
         0.00000000e+00,   3.05394932e-04,   4.01100593e-03,
         1.36596847e-04,   5.89291036e-06,   0.00000000e+00,
         4.46916984e-03,   1.56811755e-04,   5.32355781e-05,
         5.06495087e-04,   0.00000000e+00,   1.02132051e-04,
         5.67979596e-03,   1.22286763e-04,   6.21351960e-03,
         1.66952027e-02,   3.30891316e-03,   2.65500990e-02,
         4.32320130e-03,   3.46401726e-03])

## Regression Metrics

我们之前说过了用R^2来做评价指标非常不合理，所以这里我们先介绍一下常用的评价指标。

#### Explained variance score （可解释的方差变动分数）

值是0~1之间。越接近1，模型预测效果越好。

The explained_variance_score computes the explained variance regression score.

If \hat{y} is the estimated target output, y the corresponding (correct) target output, and Var is Variance, the square of the standard deviation, then the explained variance is estimated as follow:
![image1](http://scikit-learn.org/stable/_images/math/494cda4d8d05a44aa9aa20de549468e4d121e04c.png)

The best possible score is 1.0, lower values are worse.

#### Mean absolute error 
The mean_absolute_error function computes mean absolute error, a risk metric corresponding to the expected value of the absolute error loss or l1-norm loss.

If \hat{y}_i is the predicted value of the i-th sample, and y_i is the corresponding true value, then the mean absolute error (MAE) estimated over n_{\text{samples}} is defined as
![image2](http://scikit-learn.org/stable/_images/math/c38d771fb5eb121916c06cf8c651363583d17794.png)

#### Mean squared error
The mean_squared_error function computes mean square error, a risk metric corresponding to the expected value of the squared (quadratic) error loss or loss.

If \hat{y}_i is the predicted value of the i-th sample, and y_i is the corresponding true value, then the mean squared error (MSE) estimated over n_{\text{samples}} is defined as
![image](http://scikit-learn.org/stable/_images/math/44f36557fef9b30b077b21550490a1b9a0ade154.png)

#### R² score, the coefficient of determination

如果不满足统计假设的话，R^2就别用了。原因在于涉及到了平均数，而平均数又和数据分布有关。

The r2_score function computes R², the coefficient of determination. It provides a measure of how well future samples are likely to be predicted by the model. Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of y, disregarding the input features, would get a R^2 score of 0.0.

If \hat{y}_i is the predicted value of the i-th sample and y_i is the corresponding true value, then the score R² estimated over n_{\text{samples}} is defined as
![image3](http://scikit-learn.org/stable/_images/math/bdab7d608c772b3e382e2822a73ef557c80fbca2.png)

where ![image](http://scikit-learn.org/stable/_images/math/4b4e8ee0c1363ed7f781ed3a12073cfd169e3f79.png)

#### Practical Regression Metrics

就是用误差，或者precision，recall的方法来计算。

Use error metric used in classification to evaluate regression.

你回归模型建好了，但正确的比例是多少呢？如何计算，就是下面定义的函数。

errorForRegression函数有三个input：预测值，真实值，errorList
`error = np.abs(prediction - target)`计算的是误差。比如真实值是100，预测值是125，那么误差就是25.这个时候我们想知道，误差相差在10以内的我能接受，那么这一部分数据的比例是多少呢？比方说，你预测客人的价格，50，60，70，80都有可能。老板说你预测的价格和客人真实下单的价格误差在10以内的，这部分客人比例是多少，这可以作为模型的评估方法。或者说你预测的和真是的相差20元的，是多少。这部分我们能接受，就作为正确的结果，其他全设定为错误的？或你预测的增长率和实际增长率误差在0.01之内的，占比多少？

所以说这个errorList是你要填的，10元，20元，30元……这是经常用的一种指标。


In [27]:
# define the error you can bear
def errorForRegression(prediction,target,errorList):
    if not isinstance(errorList,list):
        raise TypeError("the errorList shoule be List type")
    error = np.abs(prediction - target)
    result = [np.sum(error <= x) / len(error) for x in errorList]
    return result
    

In [23]:
# Coding in here 

pred = gbr.predict(xdata)

In [24]:
ydata.describe()

count     517.000000
mean       12.847292
std        63.655818
min         0.000000
25%         0.000000
50%         0.520000
75%         6.570000
max      1090.840000
Name: area, dtype: float64

我们看到ydata的平均值是12，那么我们就设定误差在3以内的数据占比

In [28]:
result = errorForRegression(pred, ydata, [3,4,5])

In [29]:
result
# 下面的结果就是误差在3,4,5以内数据的比例

[0.14313346228239845, 0.21663442940038685, 0.35976789168278528]

这部分内容没有讲，在之后的讲座里。

## Pipeline

我们建模的时候有很多步，1标准化，2选特征，3选模型。每次都要一步步做，我们希望能把步骤提前写好，最后总的fit一下。按之前设定好的顺序慢慢执行，不用每次都要去训练一下。

Pipeline of transforms with a final estimator.

Sequentially apply a list of transforms and a final estimator. Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit.

The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters. For this, it enables setting parameters of the various steps using their names and the parameter name separated by a ‘__’, as in the example below. A step’s estimator may be replaced entirely by setting the parameter with its name to another estimator, or a transformer removed by setting to None.

In [1]:
#pipline

from sklearn import svm
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.pipeline import Pipeline

# generate some data to play with
X, y = samples_generator.make_classification(n_informative=5, n_redundant=0, random_state=42)
# ANOVA SVM-C

#先选择特征
anova_filter = SelectKBest(f_regression, k=5)
clf = svm.SVC(kernel='linear')

#创建对象，是按循序来执行的。加10个模型都没问题
anova_svm = Pipeline([('anova', anova_filter), ('svc', clf)])
# You can set the parameters using the names issued
# For instance, fit using a k of 10 in the SelectKBest
# and a parameter 'C' of the svm
# 如果上面觉得设置的不好，可以用下面这个set_parames来修改上面的参数
anova_svm.set_params(anova__k=10, svc__C=.1).fit(X, y)
                                             
# Pipeline(steps=[...])
prediction = anova_svm.predict(X)
anova_svm.score(X, y)  

# getting the selected features chosen by anova_filter
# 查看一下那些feature是被选择的。
anova_svm.named_steps['anova'].get_support()

array([False, False,  True,  True, False, False,  True,  True, False,
        True, False,  True,  True, False,  True, False,  True,  True,
       False, False], dtype=bool)

## How to choose a right estimator
[Choosing the right estimator](http://scikit-learn.org/stable/tutorial/machine_learning_map/)
![image](http://scikit-learn.org/stable/_static/ml_map.png)